In [1]:
# Download important libraries
%matplotlib inline
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
np.random.seed(42)
tf.random.set_seed(42)
import json
import os
from os import path
import tarfile
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import array_contains
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import regexp_replace, col
pipeline=None

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.1.1-bin-hadoop2.7'

In [4]:
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.0").getOrCreate()
spark

In [7]:
rev_dpath = '../data/outputs/reviews/'
        
rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)][:200]

In [11]:
def clean_df(df):
    df = df[df.caption != '']
    
    return df.reset_index(drop=True)

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
%%timeit

df_rev = pd.DataFrame()
for file in rev_files:
    
    df = pd.read_json(file)
    df_clean = clean_df(df)
    df_clean['place_id'] = file.split('_',1)[1]
    df_clean['place_id'] = df_clean['place_id'].str.replace('.json','')
    df_rev = pd.concat((df_rev, df_clean), ignore_index=True)

3.51 s ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
with open(rev_files[0]) as f:
    JSON_file = json.load(f)

JSON_file

[{'id_review': 'ChZDSUhNMG9nS0VJQ0FnSURJME52MFhBEAE',
  'caption': '',
  'relative_date': '2 years ago',
  'retrieval_date': '2021-03-23',
  'rating': 4.0,
  'username': 'MIGUEL BOSCAN',
  'n_review_user': 0,
  'n_photo_user': 0,
  'url_user': 'https://www.google.com/maps/contrib/116059736436598578008/reviews?hl=en-US',
  'census_block_group': 10730027002},
 {'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUN3a0tYS0N3EAE',
  'caption': 'Money to pawy secy y QPRyiy',
  'relative_date': '2 years ago',
  'retrieval_date': '2021-03-23',
  'rating': 1.0,
  'username': 'Prathamesh Adhikari',
  'n_review_user': 1,
  'n_photo_user': 0,
  'url_user': 'https://www.google.com/maps/contrib/114622033389765477404/reviews?hl=en-US',
  'census_block_group': 10730027002},
 {'id_review': 'ChZDSUhNMG9nS0VJQ0FnSUNBOGQ2UUh3EAE',
  'caption': '',
  'relative_date': '2 years ago',
  'retrieval_date': '2021-03-23',
  'rating': 5.0,
  'username': 'Mark Zehner',
  'n_review_user': 0,
  'n_photo_user': 0,
  'url_user': 'http

In [28]:
rev_dpath_jsonl = '../data/outputs/reviews/jsonl/'
os.makedirs(rev_dpath_jsonl, exist_ok=True)
op_file = rev_dpath_jsonl + rev_files[0].split('/reviews/')[1].split('.json')[0]
import json

with open(op_file, 'w') as outfile:
    for entry in JSON_file:
        json.dump(entry, outfile)
        outfile.write('\n')


In [30]:
file = rev_dpath_jsonl = '../data/outputs/reviews/jsonl/10730027002_ChIJNe0mPMkbiYgR6oApk2Pt7F8.json'

spark.read.json(file).head(2)

[Row(caption='', census_block_group=10730027002, id_review='ChZDSUhNMG9nS0VJQ0FnSURJME52MFhBEAE', n_photo_user=0, n_review_user=0, rating=4.0, relative_date='2 years ago', retrieval_date='2021-03-23', url_user='https://www.google.com/maps/contrib/116059736436598578008/reviews?hl=en-US', username='MIGUEL BOSCAN'),
 Row(caption='Money to pawy secy y QPRyiy', census_block_group=10730027002, id_review='ChZDSUhNMG9nS0VJQ0FnSUN3a0tYS0N3EAE', n_photo_user=0, n_review_user=1, rating=1.0, relative_date='2 years ago', retrieval_date='2021-03-23', url_user='https://www.google.com/maps/contrib/114622033389765477404/reviews?hl=en-US', username='Prathamesh Adhikari')]

In [17]:
df

DataFrame[_corrupt_record: string]